### Reduction operation: the sum of the numbers in the range [0, a)

In [20]:
import time
import sys
from numba import njit
import numpy as np

if len(sys.argv)>1:       #si el argumento que le pase por consola es mayor de 1 pues lo sumo, no tiene sentido sumar 1+0 pero si(si le pasamos 2)sumar 0+1+2
    value=int(sys.argv[1])#dado que solo vamos a coger un argumento para editar value decimos que es el [1 ]
#si hubiera que editar algo mas de nuestro codigo ese parametro seria[2]
else:
    value=1000000

def reduc_operation(a):
    """Compute the sum of the numbers in the range [0, a)."""
    x = 0
    for i in range(a):
        x += i
    return x

# Secuencial

value = 1000000

initialTime = time.time()
suma = reduc_operation(value)
finalTime = time.time()

print("Time taken by reduction operation:", (finalTime - initialTime), "seconds")

# Utilizando las operaciones mágicas de ipython
%timeit -r 2 reduc_operation(value)

print(f"\n \t Computing the sum of numbers in the range [0, value): {suma}\n")

ValueError: invalid literal for int() with base 10: '-f'

En la siguiente celda de código del notebook3 vamos a utilizar el tipo de datos de listas para
realizar dicha operación de reducción. Para ello, crea una lista de Python con la función range queontenga 106
elementos. Una vez creada, calcula el tiempo que tardas4
en sumar todos los elementosde la lista de 2 formas diferentes: 
#a) Usando un bucle for similar al código ya mostrado en la celda anterior para ir sumando al resultado total elemento a elemento de la lista, 
#b) usando la funciónsum que tienen las listas. Como verás, hay una diferencia importante

In [6]:
import time
#Usando un bucle for similar al código ya mostrado en la celda anterior para ir sumando al resultado total elemento a elemento de la lista
value=list(range(0,1000000))
start_time=time.time()
resultado=0
for i in value:
    resultado +=i
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo tomado con listas y bucles es: {diferencia}segundos")

#Usando funcion sum que tienen listas
start_time=time.time()
sum(value)
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo tomado con listas y funcion sum es: {diferencia}segundos")

El tiempo tomado con listas y bucles es: 0.5404081344604492segundos
El tiempo tomado con listas y funcion sum es: 0.03179335594177246segundos


b) En la segunda celda de código vamos a utilizar el tipo de datos array que nos proporciona el paquete
numpy. En primer lugar, pasa la lista creada en la celda anterior a un array de numpy. Y al igual que
antes, calcula el tiempo que tardas en sumar todos los elementos del array de 2 formas diferentes:
a) Usando un bucle for para ir sumando al resultado total elemento a elemento del array, y b)
usando la función sum que tiene el paquete numpy para los arrays. Como verás, hay una diferencia
importante, y también con respecto al manejo de estos datos usando listas.


In [17]:
import time

value=list(range(0,1000000))
value_numpy=np.array(value)
#sumamos todos los elementos del array usando un bucle for
start_time=time.time()
resultado=0
for i in value_numpy:
    resultado+=i
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo con for y numpy array: {diferencia}segundos")

#sumamos usando la funcion sum de numpy
start_time=time.time()
resultado=np.sum(value_numpy)
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo con sum() numpy array: {diferencia}segundos")

El tiempo con for y numpy array: 1.0051016807556152segundos
El tiempo con sum() numpy array: 0.0018012523651123047segundos


c) Crea una nueva celda de texto debajo de la última celda de código para explicar los resultados
obtenidos por los 3 procedimientos (código original, lists y numpy).

En primer lugar destacar que la solución más intuitiva de sumar números con un for, los cuales esten presentes dentro de una lista, no es la más eficiente. 
Cuando utilizamos funciones creadas por nosotros mismos como es el caso del código original, el tiempo baja considerablemente de 0.54 a 0.26754331588745117. Todo ello indica que aunque usemos bucles es mucho más óptimo controlarlos dentro de una función. 
Por otra parte crear arrays con numpy, a priori es una buena solución, pero cuando usamos un bucle para iterar los elementos del array los tiempos superan a los del bucle for con listas, mostrando que usar for con numpy no es muy adecuado a niveles de optimización.

Finalmente, el ganador indiscutible que mejora el tiempo tomado respecto al código original unas 150 veces. Por tanto, es muy conveniente el uso de funciones contenidas en la libreria numpy para realizar operaciones con array y optimizar tiempos de cómputo.

3.3. Python HPC: Numba con Jupyter notebook y uso de colas
a) Continuando con el notebook anterior, ahora vamos a usar también el paquete numba para acelerar
el tiempo de ejecución. Para ello, en la siguiente celda de código copia la celda que usaste con el
paquete numpy, y añade a cada función el decorador @njit para que ofrezca un tiempo de ejecución
menor. Como verás, hay una diferencia importante, y también con respecto a los tiempos de las
celdas anteriores
PROLOGO: njit segun he observado solo se aplica en funciones por lo que tendremos que retocar el codigo anterior y definir funciones para ambos casos

In [17]:
import time
value=list(range(0,1000000))
value_numpy=np.array(value)
#sumamos todos los elementos del array usando un bucle for
@njit
def usofor(array):
    resultado=0
    for i in array:
        resultado+=i
    return resultado
@njit
def usosum(array):
    return np.sum(array)
    
    
start_time=time.time()
resultado=usofor(value_numpy)
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo con for y numpy array @njit: {diferencia}segundos")

#sumamos usando la funcion sum de numpy
start_time=time.time()

resultado=usosum(value_numpy)
end_time=time.time()
diferencia=end_time-start_time
print(f"El tiempo con sum() numpy array @njit: {diferencia}segundos")

El tiempo con for y numpy array: 0.4989778995513916segundos
El tiempo con sum() numpy array: 0.4387533664703369segundos


En primer lugar observamos que los tiempos han ascendido considerablemente en ambos supuestos. Todo parece indicar que existe conflicto entre la optimización o procesamiento de numpy vs la compilación previa que realiza numba con las funciones seleccionadas. He buscado información sobre este conflicto y puedo decir:
Numba no puede optimizar funciones internas que posea numpy dado que es otra libreria distinta y pese que la carga para compilarla y optimizarla su interacción de carga genera overhead y vemos un declive en el tiempo de optimización, cada vez que usemos una función que este etiquetada por @njit.

